In [15]:
import cvxpy as cp
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch as torch
import argparse
import datetime

In [16]:
data=pd.read_csv('../Data/AUS Dataset2016-2019.csv')
data.rename(columns={'Heating': 'Steam_load'}, inplace=True)
data.rename(columns={'Cooling': 'Chillwater_load'}, inplace=True)
data.rename(columns={'Electricity': 'Electricity_load'}, inplace=True)
data.rename(columns={'Temperature': 'AirTemperature'}, inplace=True)
data.rename(columns={'Relative Humidity': 'DewTemperature'}, inplace=True)
def get_weekday(row):
    date = datetime.date(int(row['Year']), int(row['Month']), int(row['Day']))
    return date.weekday()  # 返回星期几的名称，例如 'Monday'
data['Week'] = data.apply(get_weekday, axis=1)
data.drop(columns=['CHWTON', 'Wind Speed','CHWTON','Wind Direction', 'Pressure', 'HTmmBTU','Precipitable Water','Dew Point'], inplace=True)

In [17]:
missing_values = data.isnull().any()

for column, has_missing in missing_values.items():
    if has_missing:
        print(f"列 {column} 有缺失值")
    else:
        print(f"列 {column} 没有缺失值")

列 Year 没有缺失值
列 Month 没有缺失值
列 Day 没有缺失值
列 Hour 没有缺失值
列 Electricity_load 没有缺失值
列 Chillwater_load 没有缺失值
列 Steam_load 没有缺失值
列 DewTemperature 没有缺失值
列 AirTemperature 没有缺失值
列 Weekday 没有缺失值
列 Week 没有缺失值


In [18]:
seq_len=4*24
label_len=24

for i in range(label_len):
    data['Electricity_load_t'+str(i)]=data['Electricity_load'].shift(-i)
for i in range(label_len):
    data['Steam_load_t'+str(i)]=data['Steam_load'].shift(-i)
for i in range(label_len):
    data['Chillwater_load_t'+str(i)]=data['Chillwater_load'].shift(-i)
for i in range(label_len):
    data['AirTemperature_t'+str(i)]=data['AirTemperature'].shift(-i)
for i in range(label_len):
    data['DewTemperature_t'+str(i)]=data['DewTemperature'].shift(-i)
for t in range(label_len):
    # 计算未来的小时数
        # 计算未来的小时数
        future_hour = (data['Hour'] + t) % 24
        future_day = data['Day'] + (data['Hour'] + t) // 24  # 计算未来的日期
        future_month = data['Month']  # 先假设月份不变，后续处理跨月情况
        future_week = (data['Week'] + (data['Hour'] + t) // 24) % 7  # 计算未来的工作日

        # 处理跨月情况
        future_month += (future_day - 1) // 31  # 假设每月最多31天
        future_day = (future_day - 1) % 31 + 1  # 确保日期在1到31之间
        future_month = (future_month - 1) % 12 + 1  # 确保月份在1到12之间

        # 将结果添加到 DataFrame
        data[f'Month_t{t}'] = future_month
        data[f'Day_t{t}'] = future_day
        data[f'Hour_t{t}'] = future_hour
        data[f'Week_t{t}'] = future_week


for i in range(seq_len):
    data['Electricity_load_t-'+str(i)]=data['Electricity_load'].shift(i+1)
for i in range(seq_len):
    data['Steam_load_t-'+str(i)]=data['Steam_load'].shift(i+1)
for i in range(seq_len):
    data['Chillwater_load_t-'+str(i)]=data['Chillwater_load'].shift(i+1)

data=data.dropna()
data=data.reset_index(drop=True)
print(data.shape)

/tmp/ipykernel_166251/2808019622.py:13: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data['DewTemperature_t'+str(i)]=data['DewTemperature'].shift(-i)
/tmp/ipykernel_166251/2808019622.py:13: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data['DewTemperature_t'+str(i)]=data['DewTemperature'].shift(-i)
/tmp/ipykernel_166251/2808019622.py:13: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd

(34945, 515)


In [19]:
data = data.drop(columns=['Year', 'Month', 'Day', 'Hour', 'Electricity_load', 'Chillwater_load',
       'Steam_load', 'DewTemperature', 'AirTemperature', 'Weekday','Week'])
data.to_csv('../Data/data.csv',index=False)